第一步：安装和导入必要包

In [ ]:
!pip install datasets
!pip install evaluate

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate

第二步：原始数据加载

In [ ]:
dataset = load_dataset("glue", "mrpc")

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [6]:
dataset['test'][0:1]

{'sentence1': ["PCCW 's chief operating officer , Mike Butcher , and Alex Arena , the chief financial officer , will report directly to Mr So ."],
 'sentence2': ['Current Chief Operating Officer Mike Butcher and Group Chief Financial Officer Alex Arena will report to So .'],
 'label': [1],
 'idx': [0]}

第三步：数据处理

In [7]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], padding="max_length", truncation=True, max_length=128)


tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

第四步：构建模型

In [12]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=2)
metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


training_args = TrainingArguments(output_dir="test_trainer",
                  eval_strategy="epoch",
                  per_device_train_batch_size=8,
                  per_device_eval_batch_size=8,
                  num_train_epochs=2,
                  logging_steps=10,
                  report_to='none',
                  )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-12-817a67fdab8a>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.739300,0.618177,0.683824
2,0.500400,0.428321,0.786765


TrainOutput(global_step=918, training_loss=0.596169224751541, metrics={'train_runtime': 57.7916, 'train_samples_per_second': 126.939, 'train_steps_per_second': 15.885, 'total_flos': 482545675530240.0, 'train_loss': 0.596169224751541, 'epoch': 2.0})

第五步：模型保存与加载

In [13]:
trainer.save_model('./my_model/')

In [14]:
model = AutoModelForSequenceClassification.from_pretrained("./my_model/")

第六步：模型推理

In [15]:
from transformers import pipeline

# 加载文本分类pipeline
classifier = pipeline("text-classification", model="./my_model/", tokenizer="google-bert/bert-base-cased", device=0)

# 输入文本进行推理
texts = {"text": "this is a good thing for me", "text_pair": "it is a good news"}
results = classifier(texts)

# 输出结果
print(results)
# for result in results:
#     print(f"Label: {result['label']}, Score: {result['score']:.4f}")

Device set to use cuda:0


{'label': 'LABEL_1', 'score': 0.8364670276641846}
